# 02 · Preparación de features (activos)


In [ ]:
# 1. KPIs presupuestos
df_kpis = crear_kpis_pptos(df)

cols_preview = [
    'CantPptos','TotPptos','CantPptosAbo','TotPptosAbo','CantPptosAvan','TotPptosAvan',
    'TicketPromPpto','PctPptosAbonados','PctPptosAvanzados','PctCumplimiento','MontoAbonadoProm'
]
[df for df in cols_preview if df in df_kpis.columns], df_kpis[cols_preview].head(3)


In [ ]:
# 1.2 Comprobación general
cols = ['CantPptos','TotPptos','CantPptosAbo','TotPptosAbo','CantPptosAvan','TotPptosAvan',
        'TicketPromPpto','PctPptosAbonados','PctPptosAvanzados','PctCumplimiento','MontoAbonadoProm']

# 1) ¿Cuántos tienen actividad de presupuestos?
print("Proporción con CantPptos notna:", df_kpis['CantPptos'].notna().mean())
print("Proporción con CantPptos > 0 :", (pd.to_numeric(df_kpis['CantPptos'], errors='coerce') > 0).mean())

# 2) Muestra SOLO pacientes con presupuesto (>0) para ver KPIs calculados
mask_act = pd.to_numeric(df_kpis['CantPptos'], errors='coerce') > 0
df_kpis.loc[mask_act, cols].sample(5, random_state=42)


In [ ]:
## 2. Transformación log1p de montos

# columnas de montos a transformar
montos = ['TotPptos', 'TotPptosAbo', 'TotPptosAvan']

df_m = df_kpis.copy()

# asegurar numérico y limpiar negativos (si los hay) -> NaN
for c in montos:
    if c in df_m.columns:
        df_m[c] = pd.to_numeric(df_m[c], errors='coerce')
        df_m.loc[df_m[c] < 0, c] = np.nan  # salvaguarda

# crear columnas transformadas con log1p
for c in montos:
    if c in df_m.columns:
        df_m[c + '_l1p'] = np.log1p(df_m[c])

# (opcional) transformar el ticket promedio si lo vas a usar en el modelo
if 'TicketPromPpto' in df_m.columns:
    df_m['TicketPromPpto_l1p'] = np.log1p(df_m['TicketPromPpto'])

# diagnóstico rápido: percentiles antes / después
def resumen_percentiles(serie):
    q = serie.quantile([0, .5, .9, .95, .99], interpolation='linear')
    q.index = ['min','p50','p90','p95','p99']
    return q

diag = {}
for c in montos:
    if c in df_m.columns and (c + '_l1p') in df_m.columns:
        diag[c] = pd.DataFrame({
            'original': resumen_percentiles(df_m[c].dropna()),
            'log1p': resumen_percentiles(df_m[c + '_l1p'].dropna())
        })

# mostrar resumen
for c, tabla in diag.items():
    print(f"\n==== {c} ====")
    display(tabla)

# vistazo de columnas creadas
cols_show = [c for c in df_m.columns if c.endswith('_l1p')][:6]
print("\nColumnas l1p creadas (ejemplo):", cols_show)
df_m[montos + cols_show].sample(5, random_state=7)


In [ ]:
# 3. Cobertura de salud unificada
df_m = unificar_cobertura_salud(df_m)
df_m['CoberturaSalud'].value_counts(dropna=False).head(10)

In [ ]:
# 4. Marcación de activos (≤ 730 días)
mask_act = marcar_activos(df_m, dias_umbral=730)
df_act = df_m[mask_act].copy()
df_inact = df_m[~mask_act].copy()
df_act.shape, df_inact.shape

In [ ]:
# 4.1 Sanity check de activos (corregido)

n_total = len(df_m)
n_act = len(df_act)
n_inact = len(df_inact)

print("Total:", n_total, " | Activos:", n_act, " | Inactivos:", n_inact)
print("Proporción activos:", round(n_act/n_total*100, 2), "%")

# Detectar columnas de atención (evitar las _pres)
ventanas = [c for c in df_m.columns if c.lower().startswith('atencion') and not c.endswith('_pres')]
print("Ventanas detectadas:", ventanas)

if ventanas:
    # Convertir columna por columna a numérico
    vnum = df_m[ventanas].apply(pd.to_numeric, errors='coerce')
    # Presencia por fila: ¿alguna ventana > 0?
    cualquier_atencion = vnum.fillna(0).gt(0).any(axis=1)
else:
    cualquier_atencion = pd.Series(False, index=df_m.index, name='cualquier_atencion')

# Regla oficial por días
dias = pd.to_numeric(df_m.get('DiasDesdeUltimaVisita'), errors='coerce')
por_dias = dias.le(730).fillna(False)

print("Activos por ventanas de atención:", int(cualquier_atencion.sum()))
print("Activos por días <= 730:", int(por_dias.sum()))

# Diferencia entre criterios (solo diagnóstico)
diff_rate = (por_dias.astype(int) - cualquier_atencion.astype(int)).abs().mean()
print("Diferencia relativa entre criterios:", round(100*diff_rate, 2), "%")




In [ ]:
# 5. Presencia de atención (ventanas 15d, 1m, 3m, 6m, 1a, 2a)

# Ventanas a usar (presencia 0/1)
ventanas_keep = ['Atencion15d','Atencion1m','Atencion3m','Atencion6m','Atencion1a','Atencion2a']

df_act = df_act.copy()

# Crear columnas *_pres
creadas = []
for v in ventanas_keep:
    if v in df_act.columns:
        # >0 => 1, NaN/<=0 => 0
        df_act[v + '_pres'] = (pd.to_numeric(df_act[v], errors='coerce') > 0).astype('uint8')
        creadas.append(v + '_pres')

print("Features de presencia creadas:", creadas)

# Cobertura por ventana (qué % de activos tiene ≥1 atención en esa ventana)
cov = df_act[creadas].mean().sort_values(ascending=False)
print("\nCobertura (proporción=1) por ventana seleccionada:\n", cov)

# (opcional) vistazo
df_act[['RutBeneficiario'] + creadas].head(3)


In [ ]:
# 5.1 Depurar presencia: eliminar redundantes (1a y 2a)
pres_cols_all = ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres',
                 'Atencion1a_pres','Atencion2a_pres']
keep_pres = ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres']

# Eliminar las redundantes si existen
drop_pres = [c for c in pres_cols_all if c not in keep_pres and c in df_act.columns]
df_act = df_act.drop(columns=drop_pres, errors='ignore')

print("Presencia mantenida:", [c for c in keep_pres if c in df_act.columns])
print("Eliminadas:", drop_pres)

# (opcional) sanity rápido
print("Cobertura final:\n", df_act[keep_pres].mean().sort_values(ascending=False).round(4))


In [ ]:
# 6. Geografía para modelar — Top-20 a partir de TODA la base (df) + whitelist de comunas clave

def _clean_comuna(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    return s.replace({'nan':'Sin Comuna','None':'Sin Comuna','':'Sin Comuna'})

# 6.0 Limpieza estándar
df_all = df.copy()
df_all['Comuna_clean'] = _clean_comuna(df_all.get('Comuna', pd.Series(index=df_all.index)))
df_act = df_act.copy()
df_act['Comuna_clean'] = _clean_comuna(df_act.get('Comuna', pd.Series(index=df_act.index)))

# 6.1 Top-20 global (sobre TODA la base)
topN = 20
top_global = df_all['Comuna_clean'].value_counts().head(topN).index.tolist()

# 6.2 Whitelist (comunas que DEBEN quedar como categoría propia aunque no queden top en activos)
whitelist = {'Iquique'}  # agrega otras si negocio lo pide, p.ej.: {'Iquique','Providencia'}

# 6.3 Lista final de categorías propias
categorias_propias = sorted(set(top_global).union(whitelist).union({'Sin Comuna'}))

# 6.4 Aplicar mapeo al subset de activos
def map_comuna_grp(val: str) -> str:
    return val if val in categorias_propias else 'Otras/Infreq'

df_act['Comuna_grp'] = df_act['Comuna_clean'].map(map_comuna_grp)

# 6.5 Región limpia
if 'Region' in df_act.columns:
    df_act['Region'] = (df_act['Region'].astype(str).str.strip()
                        .replace({'nan':'Sin Región','None':'Sin Región','':'Sin Región'}))
else:
    df_act['Region'] = 'Sin Región'

# 6.6 Tabla para revisar
top_tab = (df_act['Comuna_grp'].value_counts()
           .to_frame('n')
           .assign(pct=lambda d: (d['n']/len(df_act)).round(4)))
print("Top categorías en Comuna_grp (activos, con top global + whitelist):")
display(top_tab.head(25))

# 6.7 Dummies
df_act_dum = pd.get_dummies(df_act, columns=['Comuna_grp','Region'], drop_first=False, dtype='uint8')
dum_cols = [c for c in df_act_dum.columns if c.startswith('Comuna_grp_') or c.startswith('Region_')]
print("Total dummies geográficas creadas:", len(dum_cols))
print("Ejemplo:", dum_cols[:12])


In [ ]:
# 6b. Empresa/Convenio (features)
import pandas as pd, unicodedata, re

def _norm_text_series(s: pd.Series) -> pd.Series:
    s = s.astype(str).fillna('').str.strip()
    def _norm_one(x: str) -> str:
        x = unicodedata.normalize('NFKD', x)
        x = ''.join(c for c in x if not unicodedata.combining(c))
        return x.upper()
    return s.apply(_norm_one)

df_act = df_act.copy()
col_emp = 'Empresa'
if col_emp not in df_act.columns:
    df_act[col_emp] = ''

# Normalizar texto
df_act['Empresa_clean'] = _norm_text_series(df_act[col_emp])
df_act['Empresa_clean'] = df_act['Empresa_clean'].replace({'NAN':'', 'NONE':'', 'NULL':'', '0':''})
df_act['Empresa_clean'] = df_act['Empresa_clean'].replace('', 'SIN EMPRESA')

# Flags: tiene empresa / es convenio
df_act['TieneEmpresa'] = (df_act['Empresa_clean'] != 'SIN EMPRESA').astype('uint8')
df_act['EsConvenio']   = df_act['Empresa_clean'].str.contains(r'CONV|CONVENIO', regex=True).astype('uint8')

# Agrupar empresas: Top-15 + SIN EMPRESA + OTRAS/INFREQ
top_k = 15
vc_emp = df_act['Empresa_clean'].value_counts()
top_emp = vc_emp[vc_emp.index != 'SIN EMPRESA'].head(top_k).index.tolist()

def _map_emp(x: str) -> str:
    if x == 'SIN EMPRESA':
        return 'SIN EMPRESA'
    return x if x in top_emp else 'OTRAS/INFREQ'

df_act['Empresa_grp'] = df_act['Empresa_clean'].map(_map_emp)

# Tabla de control
emp_tab = (df_act['Empresa_grp'].value_counts()
           .to_frame('n')
           .assign(pct=lambda d: (d['n']/len(df_act)).round(4)))
print("Top Empresa_grp (activos):")
display(emp_tab.head(25))

# Incorporar a df_act_dum y hacer one-hot de Empresa_grp
df_act_dum = df_act_dum.copy()
for col in ['TieneEmpresa','EsConvenio','Empresa_grp']:
    df_act_dum[col] = df_act[col].values

df_act_dum = pd.get_dummies(df_act_dum, columns=['Empresa_grp'], drop_first=False, dtype='uint8')

dum_emp = [c for c in df_act_dum.columns if c.startswith('Empresa_grp_')]
print("Dummies Empresa creadas:", len(dum_emp))
print("Ejemplos:", dum_emp[:10])


In [ ]:
dum_comuna = [c for c in df_act_dum.columns if c.startswith('Comuna_grp_')]
dum_region = [c for c in df_act_dum.columns if c.startswith('Region_')]
print("Dummies Comuna_grp:", len(dum_comuna))
print("Dummies Region:", len(dum_region))
print("Total dummies geográficas:", len(dum_comuna) + len(dum_region))


In [ ]:
# 7 (v3). Ensamble del dataset de modelado: baseline y con Empresa/Convenio
# Requiere: df_act_dum (con dummies de geografía y, si corresponde, de empresa)

df_model = df_act_dum.copy()

# --- Selección de columnas ---
# Numéricas
num_keep = [
    'Edad',
    'CantPptos', 'CantPptosAbo', 'CantPptosAvan',
    'TotPptos_l1p', 'TotPptosAbo_l1p', 'TotPptosAvan_l1p',
    'PctCumplimiento', 'TicketPromPpto_l1p'
]

# Presencias (0/1)
pres_keep = [c for c in ['Atencion15d_pres','Atencion1m_pres','Atencion3m_pres','Atencion6m_pres']
             if c in df_model.columns]

# Dummies geográficas
dum_geo = [c for c in df_model.columns if c.startswith('Comuna_grp_') or c.startswith('Region_')]

# Señales de Empresa/Convenio
bin_emp  = [c for c in ['TieneEmpresa','EsConvenio'] if c in df_model.columns]
dum_emp  = [c for c in df_model.columns if c.startswith('Empresa_grp_')]

# Robustez: quedarnos solo con las que existen
num_keep = [c for c in num_keep if c in df_model.columns]
dum_geo  = sorted(dum_geo)
dum_emp  = sorted(dum_emp)

# --- 2 vistas: BASE (sin empresa) y EMPRESA (con empresa) ---
cols_base = num_keep + pres_keep + dum_geo
cols_emp  = cols_base + bin_emp + dum_emp

X_base = df_model[cols_base].copy()
X_emp  = df_model[cols_emp].copy() if cols_emp else None  # por si no hubiera nada de empresa

# --- Tipos y limpieza ---
def _cast_and_fill(X, num_cols, onehot_cols):
    if num_cols:
        X[num_cols] = X[num_cols].apply(pd.to_numeric, errors='coerce')
    X = X.fillna(0)
    for c in onehot_cols:
        if c in X.columns:
            X[c] = X[c].astype('uint8')
    for c in num_cols:
        if c in X.columns:
            X[c] = pd.to_numeric(X[c], downcast='float')  # float32
    return X

X_base = _cast_and_fill(X_base, num_keep, pres_keep + dum_geo)
if X_emp is not None:
    X_emp = _cast_and_fill(X_emp, num_keep, pres_keep + dum_geo + bin_emp + dum_emp)

# --- Reporte rápido ---
print("BASELINE  -> shape:", X_base.shape, "| NaNs:", int(X_base.isna().sum().sum()))
if X_emp is not None:
    print("CON EMP   -> shape:", X_emp.shape,  "| NaNs:", int(X_emp.isna().sum().sum()))
else:
    print("CON EMP   -> no se generó (no hay columnas de empresa/convenio).")

print("Grupos (BASE) -> num:", len(num_keep),
      "| pres:", len(pres_keep), "| d_geo:", len(dum_geo))
if X_emp is not None:
    print("Extra (EMP)  -> bin_emp:", len(bin_emp), "| d_emp:", len(dum_emp))

# --- Export ---
out_base = ROOT / "data" / "processed" / "activos_for_model_v2.csv"
out_base.parent.mkdir(parents=True, exist_ok=True)
X_base.to_csv(out_base, index=False, encoding="utf-8")
print("Exportado BASELINE  →", out_base.resolve())

if X_emp is not None:
    out_emp = ROOT / "data" / "processed" / "activos_for_model_v2_empresa.csv"
    out_emp.parent.mkdir(parents=True, exist_ok=True)
    X_emp.to_csv(out_emp, index=False, encoding="utf-8")
    print("Exportado CON EMP   →", out_emp.resolve())



In [ ]:
# 8 (v2). IDs para reenganchar clusters + trazabilidad (Titular y #beneficiarios)

# Base de IDs del set activo (respeta el orden del dataset modelado)
ids_v2 = (
    df_act[['RutBeneficiario']]
    .copy()
    .reset_index(drop=True)
    .rename_axis('idx_model')
    .reset_index()
)

# Enriquecimiento desde Clientes (opcional si existe el archivo)
CLI_PATH = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"
if CLI_PATH.exists():
    dfc = pd.read_csv(CLI_PATH, low_memory=False)
    if {'RutBeneficiario','RutTitular'}.issubset(dfc.columns):
        # Map RutBeneficiario -> RutTitular (1a coincidencia por beneficiario)
        map_titular = (
            dfc[['RutBeneficiario','RutTitular']]
            .dropna(subset=['RutBeneficiario'])
            .drop_duplicates(subset=['RutBeneficiario'])
            .set_index('RutBeneficiario')['RutTitular']
        )
        # Conteo de beneficiarios por titular
        ben_por_titular = (
            dfc.groupby('RutTitular')['RutBeneficiario']
            .nunique()
            .rename('Beneficiarios_por_titular')
        )
        # Enriquecer IDs
        ids_v2['RutTitular'] = ids_v2['RutBeneficiario'].map(map_titular)
        ids_v2 = ids_v2.merge(ben_por_titular, how='left',
                              left_on='RutTitular', right_index=True)
    else:
        print("Aviso: Clientes no tiene columnas RutBeneficiario y RutTitular; exporto IDs sin enriquecimiento.")
else:
    print(f"Aviso: no encuentro {CLI_PATH}; exporto IDs sin enriquecimiento.")

# Export
ids_out = ROOT / "data" / "processed" / "activos_ids_v2_plus.csv"
ids_out.parent.mkdir(parents=True, exist_ok=True)
ids_v2.to_csv(ids_out, index=False, encoding="utf-8")
print("IDs v2 PLUS exportados a:", ids_out.resolve())




